In [2]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
%matplotlib inline
plt.style.use('seaborn') #установка стиля matplotlib

In [9]:
data_csv = pd.read_csv('D:/Обучение/Skillfactory/data/new-site.csv', sep='\t')
data = pd.DataFrame(data_csv)

In [14]:
print(data.groupby('site')['dwell-time'].count())

site
0    186
1    114
Name: dwell-time, dtype: int64


In [13]:
print(data.groupby('site')['dwell-time'].sum())

site
0    16898
1    10032
Name: dwell-time, dtype: int64


In [10]:
data.head()

,site,dwell-time
0,0,2
1,1,48
2,0,15
3,1,16
4,0,38


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   site        300 non-null    int64
 1   dwell-time  300 non-null    int64
dtypes: int64(2)
memory usage: 4.8 KB


In [25]:
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import ttest_ind, shapiro, f_oneway, mannwhitneyu
from statsmodels.stats.proportion import proportions_ztest

In [23]:
converted_piv= data.groupby('site')['dwell-time'].agg(
    ['sum', 'count']
)
converted_piv

,sum,count
site,,
0,16898,186
1,10032,114


In [33]:
converted_piv['mean']=converted_piv['sum']/converted_piv['count']
converted_piv

,sum,count,mean
site,,,
0,16898,186,90.849462
1,10032,114,88.000000


In [36]:
!pip install scipy

In [37]:
!pip install statsmodels

In [44]:
data[data['site']==1]['dwell-time']

1       48
3       16
5      169
7       38
10     336
      ... 
281    187
282     11
283     70
291    100
297    144
Name: dwell-time, Length: 114, dtype: int64

In [43]:
data[data['site']==0]['dwell-time']

0        2
2       15
4       38
6      571
8      233
      ... 
294     39
295     22
296      7
298     89
299    138
Name: dwell-time, Length: 186, dtype: int64

In [38]:
from scipy.stats import shapiro

In [39]:
alpha = 0.05

In [41]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

In [45]:
_, p = shapiro(data[data['site']==1]['dwell-time'])
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.000
Данные не распределены нормально (мы отвергаем H0)


In [35]:
# нулевая и альтернативная гипотезы
H0 = 'Конверсии в группах А и B равны'
H1 = 'Конверсия в группе А выше, чем конверсия в группе B'
alpha = 0.05 # уровень значимости
# вычисляем значение p-value для z-теста для пропорций
_, p_value = proportions_ztest(
    count=converted_piv['mean'], #число "успехов"
    nobs=converted_piv['count'], #общее число наблюдений
    alternative='larger',
)
# выводим результат на экран
print('p-value: ', round(p_value, 2))
# сравниваем полученное p-value с уровнем значимости
if (p_value < alpha): 
    print("Отвергаем нулевую гипотезу. {}".format(H0))
    print("Альтернативная гипотеза. H1: {}".format(H1))
else:
    print("Принимаем нулевую гипотезу. {}".format(H0))

p-value:  1.0
Принимаем нулевую гипотезу. Конверсии в группах А и B равны


In [16]:
X = data[['R&D Spend', 'Administration', 'Marketing Spend']]

In [19]:
y = data['Profit']

In [21]:
def linear_regression(X, y):
    #Создаём вектор из единиц
    ones = np.ones(X.shape[0])
    #Добавляем вектор к таблице первым столбцом
    X = np.column_stack([ones, X])
    #Вычисляем обратную матрицу Q
    Q = np.linalg.inv(X.T @ X)
    #Вычисляем вектор коэффициентов
    w = Q @ X.T @ y
    return w

In [23]:
#Вычисляем параметры линейной регрессии
w = linear_regression(X, y)
#Выводим вычисленные значения параметров в виде вектора
print('Vector w: {}'.format(w))
#Выводим параметры с точностью до двух знаков после запятой
print('w0: {:.2f}'.format(w[0]))
print('w_R&DSpend: {:.2f}'.format(w[1]))
print('Administration: {:.2f}'.format(w[2]))
print('MarketingSpend: {:.2f}'.format(w[3]))

Vector w: [ 5.01221930e+04  8.05715050e-01 -2.68159684e-02  2.72280648e-02]
w0: 50122.19
w_R&DSpend: 0.81
Administration: -0.03
MarketingSpend: 0.03


In [24]:
y_true = [22.4, 20.6, 23.9, 22.0, 11.9]

In [25]:
y_pred = [20.5, 20.2, 20.3, 19.0, 11.0]

In [26]:
from sklearn import metrics

In [32]:
MSE = metrics.r2_score(y_true, y_pred)

In [33]:
MSE

-78.7769007731959

In [35]:
print('R2 score: {:.2f}'.format(metrics.r2_score(y_true, y_pred)))

R2 score: -78.78
